In [3]:
import pandas as pd
import vectorbt as vbt

In [5]:
# load data
start = "2020-01-01 UTC"
end = "2024-01-01 UTC"
prices = vbt.YFData.download(
    ["META", "AAPL", "AMZN", "NFLX", "GOOG"],
    start=start,
    end=end
).get("Close")

In [6]:
display(prices)

symbol,META,AAPL,AMZN,NFLX,GOOG
Date,,,,,
2020-01-02 05:00:00+00:00,208.795929,72.716072,94.900497,329.809998,68.046204
2020-01-03 05:00:00+00:00,207.691162,72.009125,93.748497,325.899994,67.712280
2020-01-06 05:00:00+00:00,211.602707,72.582924,95.143997,335.829987,69.381874
2020-01-07 05:00:00+00:00,212.060547,72.241562,95.343002,330.750000,69.338585
2020-01-08 05:00:00+00:00,214.210403,73.403648,94.598503,339.260010,69.884995
...,...,...,...,...,...
2023-12-22 05:00:00+00:00,351.732300,192.444595,153.419998,486.760010,142.047195
2023-12-26 05:00:00+00:00,353.165497,191.897858,153.410004,491.190002,142.146729
2023-12-27 05:00:00+00:00,356.151459,191.997269,153.339996,491.790009,140.773254


In [51]:
# Build the moving average indicators using VectorBT's built-in MA class
short_ma = vbt.MA.run(prices, 10, short_name="short")
long_ma = vbt.MA.run(prices, 30, short_name="long")

In [53]:
# find entry positions when the short-term moving average crosses above the long-term moving average
# Golden Cross 
entries = short_ma.ma_crossed_above(long_ma)

In [55]:
display(entries)

short_window                  10                            
long_window                   30                            
symbol                      META   AAPL   AMZN   NFLX   GOOG
Date                                                        
2020-01-02 05:00:00+00:00  False  False  False  False  False
2020-01-03 05:00:00+00:00  False  False  False  False  False
2020-01-06 05:00:00+00:00  False  False  False  False  False
2020-01-07 05:00:00+00:00  False  False  False  False  False
2020-01-08 05:00:00+00:00  False  False  False  False  False
...                          ...    ...    ...    ...    ...
2023-12-22 05:00:00+00:00  False  False  False  False  False
2023-12-26 05:00:00+00:00  False  False  False  False  False
2023-12-27 05:00:00+00:00  False  False  False  False  False
2023-12-28 05:00:00+00:00  False  False  False  False  False
2023-12-29 05:00:00+00:00  False  False  False  False  False

[1006 rows x 5 columns]

In [57]:
# set up exit positions for death crosses: when the short-term moving average drops below the long-term moving average
exits = short_ma.ma_crossed_below(long_ma)

In [59]:
# Run the backtest using the entry and exist signals
pf = vbt.Portfolio.from_signals(prices, entries, exits)

In [61]:
# visualize the average daiy return for each symbol
pf.total_return().groupby("symbol").mean().vbt.barplot()

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': '2c8d5b93-2239-43d2-b976-84410e4747d2',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([ 0.88253574,  0.47098572, -0.07520919,  1.71263501,  0.10083658])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [63]:
# see the returns for each symbol by simply holding each throughout the analysis period

(
    vbt
    .Portfolio
    .from_holding(
        prices,
        freq='1d'
    )
    .total_return()
    .groupby("symbol")
    .mean()
    .vbt
    .barplot()
)

FigureWidget({
    'data': [{'name': 'total_return',
              'showlegend': True,
              'type': 'bar',
              'uid': 'e8a95882-4ff3-40ed-8483-19b076f4714d',
              'x': array(['AAPL', 'AMZN', 'GOOG', 'META', 'NFLX'], dtype=object),
              'y': array([1.63189362, 0.60104537, 1.0613294 , 0.68729162, 0.47624392])}],
    'layout': {'height': 350,
               'legend': {'orientation': 'h',
                          'traceorder': 'normal',
                          'x': 1,
                          'xanchor': 'right',
                          'y': 1.02,
                          'yanchor': 'bottom'},
               'margin': {'b': 30, 'l': 30, 'r': 30, 't': 30},
               'template': '...',
               'width': 700}
})

In [65]:
# split the data into 4 panels
mult_prices, _ = prices.vbt.range_split(n=4)

In [67]:
display(mult_prices, _)

split_idx           0                                                 \
symbol           META        AAPL        AMZN        NFLX       GOOG   
0          208.795929   72.716072   94.900497  329.809998  68.046204   
1          207.691162   72.009125   93.748497  325.899994  67.712280   
2          211.602707   72.582924   95.143997  335.829987  69.381874   
3          212.060547   72.241562   95.343002  330.750000  69.338585   
4          214.210403   73.403648   94.598503  339.260010  69.884995   
..                ...         ...         ...         ...        ...   
246        265.837128  128.817902  160.326004  527.330017  85.768761   
247        266.852325  127.919258  159.263504  514.479980  86.210663   
248        266.145630  128.905807  158.634506  513.969971  86.532646   
249        275.700623  133.516190  164.197998  519.119995  88.385864   
250        275.481628  131.738449  166.100006  530.869995  87.521461   

split_idx           1                                                  \
symbol           META        AAPL        AMZN        NFLX        GOOG   
0          271.878632  129.609039  162.846497  540.729980   87.181076   
1          267.678436  126.405243  159.331497  522.859985   86.004646   
2          269.698883  127.968063  160.925507  520.799988   86.635651   
3          262.074799  123.660484  156.919006  500.489990   86.355476   
4          267.479370  127.880180  158.108002  508.890015   88.941231   
..                ...         ...         ...         ...         ...   
246        328.899933  172.619476  171.037003  614.239990  146.256256   
247        333.667389  173.248459  171.068497  614.090027  146.448868   
248        344.556091  177.228806  169.669495  613.119995  147.365982   
249        344.595917  176.206680  170.660995  610.710022  145.757629   
250        341.331299  176.295135  169.201004  610.539978  145.813858   

split_idx           2                                                  \
symbol           META        AAPL        AMZN        NFLX        GOOG   
0          342.744629  175.135452  168.644501  612.090027  145.314240   
1          334.772217  174.516281  166.716995  602.440002  143.997467   
2          336.951965  178.879929  170.404495  597.369995  144.390594   
3          334.951355  176.609634  167.522003  591.150024  143.735703   
4          322.649384  171.911835  164.356995  567.520020  137.004593   
..                ...         ...         ...         ...         ...   
246        119.198212  133.893219   86.769997  297.959991   89.824547   
247        116.570610  130.710220   83.790001  297.750000   87.843933   
248        117.486290  130.344513   85.250000  294.959991   89.386627   
249        116.331726  128.535507   83.040001  284.170013   87.515488   
250        115.077644  124.591377   81.820000  276.880005   86.052422   

split_idx           3                                                  
symbol           META        AAPL        AMZN        NFLX        GOOG  
0          119.775490  128.436661   84.000000  294.880005   88.311714  
1          124.154854  123.632523   85.820000  294.950012   89.277145  
2          126.772522  124.907700   85.139999  309.410004   88.291809  
3          126.344543  123.583107   83.120003  309.700012   86.360947  
4          129.410080  128.130234   86.080002  315.549988   87.744400  
..                ...         ...         ...         ...         ...  
246        351.732300  192.444595  153.419998  486.760010  142.047195  
247        353.165497  191.897858  153.410004  491.190002  142.146729  
248        356.151459  191.997269  153.339996  491.790009  140.773254  
249        356.639160  192.424698  153.380005  490.510010  140.613998  
250        352.299622  191.380966  151.940002  486.880005  140.265640  

[251 rows x 20 columns]

[DatetimeIndex(['2020-01-02 05:00:00+00:00', '2020-01-03 05:00:00+00:00',
                '2020-01-06 05:00:00+00:00', '2020-01-07 05:00:00+00:00',
                '2020-01-08 05:00:00+00:00', '2020-01-09 05:00:00+00:00',
                '2020-01-10 05:00:00+00:00', '2020-01-13 05:00:00+00:00',
                '2020-01-14 05:00:00+00:00', '2020-01-15 05:00:00+00:00',
                ...
                '2020-12-15 05:00:00+00:00', '2020-12-16 05:00:00+00:00',
                '2020-12-17 05:00:00+00:00', '2020-12-18 05:00:00+00:00',
                '2020-12-21 05:00:00+00:00', '2020-12-22 05:00:00+00:00',
                '2020-12-23 05:00:00+00:00', '2020-12-24 05:00:00+00:00',
                '2020-12-28 05:00:00+00:00', '2020-12-29 05:00:00+00:00'],
               dtype='datetime64[ns, UTC]', name='split_0', length=251, freq=None),
 DatetimeIndex(['2020-12-31 05:00:00+00:00', '2021-01-04 05:00:00+00:00',
                '2021-01-05 05:00:00+00:00', '2021-01-06 05:00:00+00:00',
       

In [89]:
# with each different panel, we can run different combinations of our short and long moving average windows
short_ma = vbt.MA.run(mult_prices, [10, 20], short_name="short")
long_ma = vbt.MA.run(mult_prices, [30, 50], short_name="long")

In [103]:
# find the entries and exits
entries = short_ma.ma_crossed_above(long_ma)
exits = short_ma.ma_crossed_below(long_ma)
display(entries)
display(exits)

short_window     10                                                          \
long_window      30                                                           
split_idx         0                                  1                        
symbol         META   AAPL   AMZN   NFLX   GOOG   META   AAPL   AMZN   NFLX   
0             False  False  False  False  False  False  False  False  False   
1             False  False  False  False  False  False  False  False  False   
2             False  False  False  False  False  False  False  False  False   
3             False  False  False  False  False  False  False  False  False   
4             False  False  False  False  False  False  False  False  False   
..              ...    ...    ...    ...    ...    ...    ...    ...    ...   
246           False  False  False  False  False  False  False  False  False   
247           False  False  False  False  False  False  False  False  False   
248           False  False  False  False  False  False  False  False  False   
249           False  False  False  False  False  False  False  False  False   
250           False  False  False  False  False  False  False  False  False   

short_window         ...     20                                            \
long_window          ...     50                                             
split_idx            ...      2                                  3          
symbol         GOOG  ...   META   AAPL   AMZN   NFLX   GOOG   META   AAPL   
0             False  ...  False  False  False  False  False  False  False   
1             False  ...  False  False  False  False  False  False  False   
2             False  ...  False  False  False  False  False  False  False   
3             False  ...  False  False  False  False  False  False  False   
4             False  ...  False  False  False  False  False  False  False   
..              ...  ...    ...    ...    ...    ...    ...    ...    ...   
246           False  ...  False  False  False  False  False  False  False   
247           False  ...  False  False  False  False  False  False  False   
248           False  ...  False  False  False  False  False  False  False   
249           False  ...  False  False  False  False  False  False  False   
250           False  ...  False  False  False  False  False  False  False   

short_window                       
long_window                        
split_idx                          
symbol         AMZN   NFLX   GOOG  
0             False  False  False  
1             False  False  False  
2             False  False  False  
3             False  False  False  
4             False  False  False  
..              ...    ...    ...  
246           False  False  False  
247           False  False  False  
248           False  False  False  
249           False  False  False  
250           False  False  False  

[251 rows x 40 columns]

short_window     10                                                          \
long_window      30                                                           
split_idx         0                                  1                        
symbol         META   AAPL   AMZN   NFLX   GOOG   META   AAPL   AMZN   NFLX   
0             False  False  False  False  False  False  False  False  False   
1             False  False  False  False  False  False  False  False  False   
2             False  False  False  False  False  False  False  False  False   
3             False  False  False  False  False  False  False  False  False   
4             False  False  False  False  False  False  False  False  False   
..              ...    ...    ...    ...    ...    ...    ...    ...    ...   
246           False  False  False  False  False  False  False  False  False   
247           False  False  False  False  False  False  False  False  False   
248           False  False  False  False  False  False  False  False  False   
249           False  False  False  False  False  False  False  False  False   
250           False  False  False  False  False  False  False  False  False   

short_window         ...     20                                            \
long_window          ...     50                                             
split_idx            ...      2                                  3          
symbol         GOOG  ...   META   AAPL   AMZN   NFLX   GOOG   META   AAPL   
0             False  ...  False  False  False  False  False  False  False   
1             False  ...  False  False  False  False  False  False  False   
2             False  ...  False  False  False  False  False  False  False   
3             False  ...  False  False  False  False  False  False  False   
4             False  ...  False  False  False  False  False  False  False   
..              ...  ...    ...    ...    ...    ...    ...    ...    ...   
246           False  ...  False  False  False  False  False  False  False   
247           False  ...  False  False  False  False  False  False  False   
248           False  ...  False  False  False  False  False  False  False   
249           False  ...  False  False  False  False  False  False  False   
250           False  ...  False  False  False  False  False  False  False   

short_window                       
long_window                        
split_idx                          
symbol         AMZN   NFLX   GOOG  
0             False  False  False  
1             False  False  False  
2             False  False  False  
3             False  False  False  
4             False  False  False  
..              ...    ...    ...  
246           False  False  False  
247           False  False  False  
248           False  False  False  
249           False  False  False  
250           False  False  False  

[251 rows x 40 columns]

In [105]:
# backtest the different combinations
pf = vbt.Portfolio.from_signals(
    mult_prices,
    entries,
    exits,
    freq="1D"
)

In [107]:
# visualize the results by grouping total returns by split index and symbol, finding the mean, and plotting
(
    pf
    .total_return()
    .groupby(
        ['split_idx', 'symbol']
    )
    .mean()
    .unstack(level=-1)
    .vbt
    .barplot()
)

FigureWidget({
    'data': [{'name': 'AAPL',
              'showlegend': True,
              'type': 'bar',
              'uid': 'ce3883e4-de15-4b7e-bbde-72737dff17d2',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.65421467,  0.250877  , -0.1508245 ,  0.07862499])},
             {'name': 'AMZN',
              'showlegend': True,
              'type': 'bar',
              'uid': 'ffd1f5e0-b27a-4bb0-b58b-9deb83f18eec',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.44307246, -0.08539409, -0.13758284,  0.30240588])},
             {'name': 'GOOG',
              'showlegend': True,
              'type': 'bar',
              'uid': '3fe2a6c0-4209-471e-8a16-8635aa6b8b38',
              'x': array([0, 1, 2, 3], dtype=int64),
              'y': array([ 0.21019083,  0.11582478, -0.27358192,  0.1385438 ])},
             {'name': 'META',
              'showlegend': True,
              'type': 'bar',
              'uid': 'c0cf41

In [109]:
# get trading statistics from the backtest analysis
pf.orders.stats(group_by=True)

Start                                0
End                                250
Period               251 days 00:00:00
Total Records                      234
Total Buy Orders                   129
Total Sell Orders                  105
Min Size                      0.158054
Max Size                      1.591699
Avg Size                      0.637744
Avg Buy Size                  0.639425
Avg Sell Size                 0.635679
Avg Buy Price               220.763887
Avg Sell Price                221.4146
Total Fees                         0.0
Min Fees                           0.0
Max Fees                           0.0
Avg Fees                           0.0
Avg Buy Fees                       0.0
Avg Sell Fees                      0.0
Name: group, dtype: object

In [111]:
# extract the sharpe ratio for each combination of split
pf.sharpe_ratio()

short_window  long_window  split_idx  symbol
10            30           0          META      1.233466
                                      AAPL      2.062475
                                      AMZN      1.929362
                                      NFLX      0.157717
                                      GOOG      1.094471
                           1          META      0.963191
                                      AAPL      2.039327
                                      AMZN      0.064464
                                      NFLX      0.354068
                                      GOOG      1.851290
                           2          META     -1.300484
                                      AAPL     -0.477565
                                      AMZN     -0.541624
                                      NFLX     -0.245790
                                      GOOG     -2.348113
                           3          META      0.248419
                                      AAPL 